In [9]:

import pandas as pd
import multiprocessing
import re
from tqdm import tqdm
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer
import torch
from torch.nn.utils.rnn import pad_sequence
from datasets import Dataset
from preprocess import TextPreprocessor, replace_text_with_ocr, split_text_column
from merge import merge_data
from dataset_creation import split_data
from preprocess import *
from proccessclaims import *
logging.basicConfig(level=logging.DEBUG)
from datasets import Dataset
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import *

from unsloth import FastLanguageModel
import torch
import pandas as pd
import logging
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import Dataset
from preprocess import TextPreprocessor, replace_text_with_ocr, split_text_column
from merge import merge_data
from dataset_creation import split_data
from proccessclaims import create_pos_neg_pairs
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from peft import LoraConfig, get_peft_model
import os
import json
import pickle

In [10]:



logging.basicConfig(level=logging.DEBUG)

def pre_process():
    logging.debug("Starting data preprocessing.")

    fact_checks = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/fact_checks.csv")
    posts = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/posts.csv")
    pairs = pd.read_csv("/home/stud/haroonm0/localdisk/Fact_check/dataset/pairs.csv")
    preprocessor = TextPreprocessor()

    if 'text' in posts.columns and 'ocr' in posts.columns:
        posts = replace_text_with_ocr(posts, text_column='text', ocr_column='ocr')
    if 'ocr' in posts.columns:
        posts[['ocr_original', 'ocr_translated', 'ocr_language', 'ocr_confidence']] = posts.apply(
            lambda row: split_text_column(row, 'ocr'), axis=1
        )
    if 'text' in posts.columns:
        posts[['text_original', 'text_translated', 'text_language', 'text_confidence']] = posts.apply(
            lambda row: split_text_column(row, 'text'), axis=1
        )
    fact_checks[['original_claim', 'translated_claim', 'language', 'confidence']] = fact_checks.apply(
        lambda row: split_text_column(row, 'claim'), axis=1
    )

    columns_to_preprocess = [
        ('translated_claim', fact_checks),
        ('text_translated', posts),
        ('ocr_translated', posts),
        ('original_claim', fact_checks),
        ('text_original', posts),
        ('ocr_original', posts)
    ]
    for col, df in columns_to_preprocess:
        if col in df.columns:
            df[col] = df[col].apply(preprocessor.preprocess)

    mergedata = merge_data(posts, fact_checks, pairs)
    mergedata = mergedata.drop_duplicates(subset="translated_claim", keep="first")
    mergedata = mergedata.drop_duplicates(subset="original_claim", keep="first")
    train_df, val_df, test_df = split_data(posts, mergedata)

    return train_df, test_df, val_df, mergedata


In [14]:
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments
from peft import LoraConfig, get_peft_model
from trl import SFTTrainer
import pandas as pd
import os
from transformers import BitsAndBytesConfig



def tokenize_ranking_data(examples, tokenizer, max_seq_length):
    # Format the texts
    formatted_texts = [
        f"Post: {post}\nClaim: {claim}" for post, claim in zip(examples["post"], examples["claim"])
    ]

    # Tokenize the batch
    tokenized = tokenizer(
        formatted_texts,
        max_length=max_seq_length,
        truncation=True,
        padding="max_length"
    )

    # Add labels
    tokenized["labels"] = examples["label"]
    return tokenized


def save_dataframe_to_csv(df, filename):
    """Save a DataFrame as a CSV file."""
    df.to_csv(filename, index=False)


def load_csv_to_dataframe(filename):
    """Load a CSV file into a DataFrame."""
    if not os.path.exists(filename) or os.path.getsize(filename) == 0:
        return None
    return pd.read_csv(filename)


def format_ranking_data_from_pairs(pairs):
    formatted_data = []
    for _, row in pairs.iterrows():
        formatted_data.append({
            "post": str(row["text_original"]),
            "claim": str(row["original_claim"]),
            "label": int(row["label"])
        })
    return formatted_data

# Preprocess data
train_df, test_df, val_df, mergedata = pre_process()

# File paths
train_file = "train_pairs.json"
val_file = "val_pairs.json"
test_file = "test_pairs.json"

# Try loading data, else generate it
train_pairs = load_csv_to_dataframe(train_file)
if train_pairs is None:
    train_pairs = create_pos_neg_pairs(train_df, mergedata)
    save_dataframe_to_csv(train_pairs, train_file)

val_pairs = load_csv_to_dataframe(val_file)
if val_pairs is None:
    val_pairs = create_pos_neg_pairs(val_df, mergedata)
    save_dataframe_to_csv(val_pairs, val_file)

test_pairs = load_csv_to_dataframe(test_file)
if test_pairs is None:
    test_pairs = create_pos_neg_pairs(test_df, mergedata)
    save_dataframe_to_csv(test_pairs, test_file)

# Format data for ranking
train_data = format_ranking_data_from_pairs(train_pairs)
val_data = format_ranking_data_from_pairs(val_pairs)
test_data = format_ranking_data_from_pairs(test_pairs)

train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)


max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.


model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit"
llm_int8_enable_fp32_cpu_offload=True
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    llm_int8_enable_fp32_cpu_offload=llm_int8_enable_fp32_cpu_offload,
)

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_name,
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    quantization_config=bnb_config,  # Pass the quantization config
)


model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Define the prompt template for retrieval task
retrieval_prompt = """Below is a post and a claim. Indicate if the claim correctly aligns with the post. Use 1 for correct and 0 for incorrect.

### Post:
{}

### Claim:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token  # Ensure the EOS token is used to mark the end of sequences.

def formatting_func(examples):
    # Extract fields from the dataset
    posts = examples.get("post", [])
    claims = examples.get("claim", [])
    labels = examples.get("label", [])
    
    # Ensure all fields are lists
    if not isinstance(posts, list):
        posts = [posts]
    if not isinstance(claims, list):
        claims = [claims]
    if not isinstance(labels, list):
        labels = [labels]
    
    # Format the texts
    texts = []
    for post, claim, label in zip(posts, claims, labels):
        text = retrieval_prompt.format(post, claim, label) + EOS_TOKEN
        texts.append(text)
    
    return {"text": texts}


max_seq_length = 2048
train_dataset = train_dataset.map(formatting_func, batched=True)
val_dataset = val_dataset.map(formatting_func, batched=True)
test_dataset = test_dataset.map(formatting_func, batched=True)

trainer = SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    dataset_text_field="text",  # Use the 'text' column
    max_seq_length=max_seq_length,
    formatting_func=None,  # Formatting is already done
    args=TrainingArguments(
    per_device_train_batch_size=16,  # Reduce batch size
    gradient_accumulation_steps=16,  # Compensate for reduced batch size
    warmup_steps=100,
    num_train_epochs=3,
    learning_rate=1e-5,
    fp16=False,
    bf16=True,
    logging_steps=10,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    evaluation_strategy="steps",  # Evaluate during training
    eval_steps=500,  # Evaluate every 50 steps
    save_steps=500,  # Save model checkpoints every 50 steps
    save_total_limit=2,  # Keep only the last 2 checkpoints
    load_best_model_at_end=True,  # Load the best model at the end
    logging_dir="./logs",  # Directory for logs
    logging_strategy="steps",
    report_to="none",
)
)



DEBUG:root:Starting data preprocessing.


DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): raw.githubusercontent.com:443
DEBUG:urllib3.connectionpool:https://raw.githubusercontent.com:443 "GET /unslothai/unsloth/main/unsloth/models/mapper.py HTTP/11" 200 2595
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/adapter_config.json HTTP/11" 404 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/commits/main HTTP/11" 200 14204
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /api/models/unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/commits/main?p=1 HTTP/11" 200 5613
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unslothai/other/resolve/mai

==((====))==  Unsloth 2024.12.8: Fast Llama patching. Transformers: 4.46.3.
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.553 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1. CUDA: 8.9. CUDA Toolkit: 12.1. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GET /unslothai/other/resolve/main/config.json HTTP/11" 200 635
DEBUG:filelock:Attempting to release lock 139663788903680 on /home/stud/haroonm0/.cache/huggingface/hub/.locks/models--unslothai--other/c0fd5260818f672a9119328c53cc33df84f30698.lock
DEBUG:filelock:Lock 139663788903680 released on /home/stud/haroonm0/.cache/huggingface/hub/.locks/models--unslothai--other/c0fd5260818f672a9119328c53cc33df84f30698.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unslothai/other/resolve/main/config.json HTTP/11" 200 0
DEBUG:filelock:Attempting to acquire lock 139663788911552 on /home/stud/haroonm0/.cache/huggingface/hub/.locks/models--unslothai--other/c0fd5260818f672a9119328c53cc33df84f30698.lock
DEBUG:filelock:Lock 139663788911552 acquired on /home/stud/haroonm0/.cache/huggingface/hub/.locks/models--unslothai--other/c0fd5260818f672a9119328c53cc33df84f30698.lock
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "GE

In [15]:
model = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 45,544 | Num Epochs = 10
O^O/ \_/ \    Batch size per device = 4 | Gradient Accumulation steps = 8
\        /    Total batch size = 32 | Total steps = 14,230
 "-____-"     Number of trainable parameters = 41,943,040


Step,Training Loss,Validation Loss
50,2.512300,2.331392
100,2.125100,2.033740
150,2.028300,1.992980
200,2.017500,1.956154
250,1.988800,1.918028
300,1.962900,1.906777
350,1.925600,1.900904
400,1.914000,1.895901
450,1.992400,1.891156
500,1.880200,1.887724


DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:Resetting dropped connection: huggingface.co
DEBUG:urllib3.connectionpool:https://huggingface.co:443 "HEAD /unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit/resolve/main/config.json HTTP/11" 200 0
DEBUG:urllib3.connectionpool:https://huggin

KeyboardInterrupt: 